# Carga Gold - Fato Carteira

Este notebook realiza a carga da fato de cotação (fato_cotacao) a partir dos dados da tabela staging de cotação histórica.

## Imports

In [1]:
from spark_config import init_spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window

## Start Spark Session

In [2]:
spark = init_spark("Carga fato carteira")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Define Delta Table Paths

In [3]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_controle_ativo = f"{base_silver_path}/stg_controle_ativo"
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"
delta_path_fato_carteira = f"{base_gold_path}/fato_carteira"

In [4]:
"""
Pipeline para carregar posição mensal de carteira de investimentos
Abordagem funcional
"""
def extrair_operacoes_mes(spark: SparkSession, mes_ref_base: str, cpf: str = None) -> DataFrame:
    """
    Extrai operações do mês da tabela silver
    
    Args:
        spark: SparkSession
        mes_ref_base: Mês de referência base no formato 'YYYY-MM'
        cpf: CPF do investidor (opcional)
    """
    query = f"""
    SELECT 
        mes_referencia,
        '{mes_ref_base}' AS mes_ref_base,
        cpf,
        cotista,
        cd_ativo,
        dt_operacao,
        cd_tipo_operacao,
        qt_operacao,
        vl_preco_ativo,
        vl_custo_total,
        vl_rateio,
        vl_liquido,
        qt_estoque,
        vl_pmedio,
        vl_ganho_perda,
        ir_mes,
        vl_vendas_mes,
        ts_insercao
    FROM silver.stg_controle_ativo
    WHERE mes_referencia <= '{mes_ref_base}'
    """
    
    if cpf:
        query += f" AND cpf = '{cpf}'"
        
    return spark.sql(query)


def extrair_cotacao_mes(spark: SparkSession, mes_ref_base: str) -> DataFrame:
    """
    Extrai a última cotação do mês da tabela gold
    
    Args:
        spark: SparkSession
        mes_ref_base: Mês de referência base no formato 'YYYY-MM'
    """
    query = f"""
    SELECT 
        t1.dt_pregao,
        t2.cd_ativo,
        t1.vl_medio,
    FROM gold.fato_cotacao t1
    INNER JOIN gold.dim_ativo_financeiro t2 ON t1.sk_ativo = t2.sk_ativo)
    WHERE mes_referencia <= '{mes_ref_base}'
    """
          
    return spark.sql(query)



def calcular_posicao_final(df_operacoes: DataFrame) -> DataFrame:
    """
    Calcula a posição final de cada ativo no mês.
    Considera múltiplas operações no mesmo mês e ajusta estoque conforme tipo de operação.
    """
    # Ajusta a quantidade: vendas como negativas
    df_ajustado = df_operacoes.withColumn(
        "qt_ajustada",
        F.when(F.col("cd_tipo_operacao").isin("V", "VENDA"), -F.col("qt_operacao"))
         .otherwise(F.col("qt_operacao"))
    ).withColumn(
        "vl_total_operacao_ajustada",
        (F.col("qt_ajustada") * F.col("vl_preco_ativo")) + F.col("vl_rateio")
    )

    # Agrupa por ativo, cotista e mês de referência
    df_agrupado = df_ajustado.groupBy(
        "mes_ref_base",
        "cpf",
        "cotista",
        "cd_ativo"
    ).agg(
        F.sum("qt_ajustada").alias("qt_estoque"),
        (F.sum("vl_total_operacao_ajustada") / F.sum("qt_ajustada")).alias("vl_pmedio"),
    )

    # Calcula o valor da posição
    df_resultado = df_agrupado.withColumn(
        "vl_investido", F.col("qt_estoque") * F.col("vl_pmedio")
    )

    return df_resultado


def agregar_por_cotista(df_posicao: DataFrame) -> DataFrame:
    """
    Agrega métricas por cotista
    """
    return df_posicao.groupBy("mes_ref", "cpf", "cotista").agg(
        F.count("cd_ativo").alias("qt_ativos_carteira"),
        F.sum("qt_estoque").alias("qt_total_acoes"),
        F.sum("vl_posicao").alias("vl_total_carteira"),
        F.sum("vl_ganho_perda").alias("vl_total_ganho_perda"),
        F.sum("ir_mes").alias("vl_total_ir_mes"),
        F.sum("vl_vendas_mes").alias("vl_total_vendas_mes"),
        F.avg("vl_pmedio").alias("vl_pmedio_ponderado")
    )


In [6]:
# 1. Extrair operações
mes_referencia = "2025-02"
cpf = None  # Ou defina um CPF específico para filtrar

print("1. Extraindo operações da silver...")
df_operacoes = extrair_operacoes_mes(spark, mes_referencia, cpf)
count_operacoes = df_operacoes.count()
print(f"   → {count_operacoes} operações encontradas")

if count_operacoes == 0:
    print("⚠ Nenhuma operação encontrada para o período")
    
# 2. Calcular posição final por ativo
print("2. Calculando posição final por ativo...")
df_posicao = calcular_posicao_final(df_operacoes)
print(f"   → {df_posicao.count()} ativos na carteira")

df_posicao.orderBy("cd_ativo", "cpf").show(15)


1. Extraindo operações da silver...
   → 54 operações encontradas
2. Calculando posição final por ativo...
   → 10 ativos na carteira
+------------+-----------+-------+--------+----------+-----------+------------+
|mes_ref_base|        cpf|cotista|cd_ativo|qt_estoque|  vl_pmedio|vl_investido|
+------------+-----------+-------+--------+----------+-----------+------------+
|     2025-02|36725979882|TAYNARA|  AFHI11|        38| 92.0615789| 3498.339998|
|     2025-02|36725979882|TAYNARA|  BTLG11|        20| 95.4160000| 1908.320000|
|     2025-02|36725979882|TAYNARA|  CPTI11|         8| 77.0000000|  616.000000|
|     2025-02|36725979882|TAYNARA|  IRDM11|        34| 64.7826471| 2202.610001|
|     2025-02|36725979882|TAYNARA|  KNSC11|       242|  8.6740909| 2099.129998|
|     2025-02|36725979882|TAYNARA|  RZTR11|        22| 88.2468182| 1941.430000|
|     2025-02|36725979882|TAYNARA|  TRBL11|        38| 70.7163158| 2687.220000|
|     2025-02|36725979882|TAYNARA|  TRXF11|        30|102.5983333|

## Read Source Data

In [ ]:
# Lê a tabela staging de cotação histórica
df_stg_controle_ativo = spark.read.format("delta").load(delta_path_controle_ativo)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_controle_ativo}")
print(f"Total de registros: {df_stg_controle_ativo.count()}")

## Transform Data

In [ ]:
# Filtra apenas registros com tp_mercado = '10'
df_stg_cotacao_historica = df_stg_cotacao_historica.filter(col("tp_mercado") == "010")

# Join LEFT (mantém todas as cotações mesmo sem correspondência na dimensão)
df_joined = (
    df_stg_cotacao_historica.alias("stg")
    .join(
        df_dim_ativo.alias("dim"),
        col("stg.cd_negociacao") == col("dim.cd_ativo"),
        "left"  # mantém as linhas da stg mesmo se não achar na dimensão
    )
)

# Tratamento da chave surrogate faltante (usa -1)
df_fato_cotacao = (
    df_joined.select(
        col("stg.dt_pregao"),
        when(col("sk_ativo").isNull(), lit("-1")).otherwise(col("sk_ativo")).alias("sk_ativo"),
        col("vl_abertura"),
        col("vl_minimo"),
        col("vl_maximo"),
        col("vl_medio"),
        col("vl_ultimo_negocio"),
        col("qt_negocios_efetuados").alias("qt_negocio"),
        col("qt_total_titulos").alias("qt_titulo"),
        col("vl_total_titulos").alias("vl_volume"),
        current_timestamp().alias("ts_insercao"),
        year(col("dt_pregao")).alias("ano_pregao"),
        month(col("dt_pregao")).alias("mes_pregao")
    )
)

## Write Data

In [ ]:

df_fato_cotacao.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "false") \
    .option("partitionOverwriteMode", "dynamic") \
    .partitionBy("ano_pregao", "mes_pregao") \
    .save(delta_path_fato_cotacao)

print("✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas")

## Stop Spark Session

In [17]:
# Encerra a SparkSession
spark.stop()